In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [6]:
URL = "https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile"
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

In [9]:
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name = driver.find_elements(By.CLASS_NAME, "travelsName___61bd75")
            bus_type = driver.find_elements(By.CLASS_NAME, "busType___44b170")
            departing_time = driver.find_elements(By.CLASS_NAME, "boardingTime___6bc587")
            duration = driver.find_elements(By.CLASS_NAME, "duration___da3927")
            reaching_time = driver.find_elements(By.CLASS_NAME, "droppingTime___a452f5")
            star_rating = driver.find_elements(By.XPATH, "rating___f7ba20")
            price = driver.find_elements(By.CLASS_NAME, "finalFare___c05744")

            # Use XPath to handle both seat availability classes
            seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'totalSeats___5f5f18') or contains(@class, 'seatsWrap___1a5212')]")

            bus_details = []
            for i in range(len(bus_name)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name[i].text,
                    "Bus_Type": bus_type[i].text,
                    "Departing_Time": departing_time[i].text,
                    "Duration": duration[i].text,
                    "Reaching_Time": reaching_time[i].text,
                    "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                    "Price": price[i].text,
                    "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

driver = None

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 5):  # There are 4 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('bihar.csv', index=False)

 #Close the driver
if driver is not None:
    driver.quit()

Error occurred while accessing page 3: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff6b2d0fe75+79173]
	GetHandleVerifier [0x0x7ff6b2d0fed0+79264]
	(No symbol) [0x0x7ff6b2ac9e5a]
	(No symbol) [0x0x7ff6b2b20586]
	(No symbol) [0x0x7ff6b2b2083c]
	(No symbol) [0x0x7ff6b2b74247]
	(No symbol) [0x0x7ff6b2b489af]
	(No symbol) [0x0x7ff6b2b7100d]
	(No symbol) [0x0x7ff6b2b48743]
	(No symbol) [0x0x7ff6b2b114c1]
	(No symbol) [0x0x7ff6b2b12253]
	GetHandleVerifier [0x0x7ff6b2fda2ad+3004797]
	GetHandleVerifier [0x0x7ff6b2fd46fd+2981325]
	GetHandleVerifier [0x0x7ff6b2ff3350+3107360]
	GetHandleVerifier [0x0x7ff6b2d2a9fe+188622]
	GetHandleVerifier [0x0x7ff6b2d3228f+219487]
	GetHandleVerifier [0x0x7ff6b2d18dc4+115860]
	GetHandleVerifier [0x0x7ff6b2d18f79+116297]
	GetHandleVerifier [0x0x7ff6b2cff528+11256]
	BaseThreadInitThunk [0x0x7ffa2bb7257d+29]
	RtlUserThreadStart [0x0x7ffa2c54af08+40]

Error occurred while accessing page 4: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff6b2d0fe75+79173]
	GetHandle

In [10]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,V.I.P Travels,A/C Seater / Sleeper (2+2),21:40,6h,03:40,0,₹350,33 Seats
1,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Jai Hanuman Travels,A/C Seater / Sleeper (2+2),22:20,7h 10m,05:30,0,₹300,40 Seats
2,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Jai Mata Di Tour and Travels,A/C Seater / Sleeper (2+2),22:40,5h 20m,04:00,0,₹405,4 Seats
3,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Om Sai Ram Bus,NON A/C Seater (2+2),17:55,4h 25m,22:20,0,₹350,41 Seats
4,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Om Sai Ram Bus,A/C Seater (2+2),18:10,4h 20m,22:30,0,₹400,41 Seats
...,...,...,...,...,...,...,...,...,...,...
127,Patna (Bihar) to Araria (Bihar),https://www.redbus.in/bus-tickets/patna-to-araria,Krishna City and Cargo,A/C Seater / Sleeper (2+1),19:00,9h,04:00,0,₹495,19 Seats\n(4 Single)
128,Patna (Bihar) to Araria (Bihar),https://www.redbus.in/bus-tickets/patna-to-araria,MAA SHANTI TRAVELS,A/C Seater / Sleeper (2+1),21:25,8h 5m,05:30,0,₹630,6 Seats
129,Patna (Bihar) to Araria (Bihar),https://www.redbus.in/bus-tickets/patna-to-araria,Maa Durga Rath,A/C Seater / Sleeper (2+1),20:00,8h,04:00,0,₹500,22 Seats
130,Patna (Bihar) to Araria (Bihar),https://www.redbus.in/bus-tickets/patna-to-araria,Sadhvi Rath,A/C Seater / Sleeper (2+1),20:20,8h 50m,05:10,0,₹558,7 Seats\n(1 Single)
